In [1]:
import sys
import os
import numpy as np
import multiprocessing
import pickle
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
from statsmodels.api import OLS, add_constant
import time
from scipy.stats import norm
import datetime as dt

from vax_sims_LHS_samples import *
from plot_utils import *

In [2]:
UNCERTAINTY_PARAMS = ['vax_susc_mult', 'vax_transmission_mult', 'contacts_per_day_mult', 'outside_infection_rate_mult',
                      'cases_isolated_per_contact_trace', 'initial_ID_prevalence']

UNCERTAINTY_PARAM_RANGES = {
    'vax_susc_mult': (0.097608, 0.941192), # 0.5194 +/- 1.96 * 0.2152
    'vax_transmission_mult': (0.25, 1),
    'contacts_per_day_mult': (0.9,2.7),
    'outside_infection_rate_mult': (1, 5),
    'cases_isolated_per_contact_trace': (0.5,1.5),
    'initial_ID_prevalence': (0.003, 0.0054)
}

def sample_from_prior():
    return_point = list()
    for param in UNCERTAINTY_PARAMS:
        mean = np.mean(UNCERTAINTY_PARAM_RANGES[param])
        sd = (UNCERTAINTY_PARAM_RANGES[param][1] - UNCERTAINTY_PARAM_RANGES[param][0])/(2*1.96)
        return_point.append(np.random.normal(mean, sd))
    return return_point

In [3]:
def map_lhs_point_to_vax_sim(lhs_point, param_modifiers=None):
    base_params, base_group_names, contact_matrix, vax_rates = load_calibrated_params()

    vax_susc_mult = lhs_point[0]
    vax_trans_mult = lhs_point[1]

    contact_matrix = contact_matrix * lhs_point[2]

    for params in base_params:
        params['daily_outside_infection_p'] = params['daily_outside_infection_p'] * lhs_point[3]


    base_params[0]['cases_isolated_per_contact'] *= lhs_point[4]
    base_params[1]['cases_isolated_per_contact'] *= lhs_point[4]
    base_params[2]['cases_isolated_per_contact'] *= lhs_point[4]
    
    base_params[0]['initial_ID_prevalence'] = lhs_point[5]
    base_params[1]['initial_ID_prevalence'] = lhs_point[5]
    base_params[2]['initial_ID_prevalence'] = lhs_point[5]
    
    vax_sim = generate_vax_unvax_multigroup_sim(base_params, base_group_names,
                                    vax_rates, contact_matrix,
                                    vax_trans_mult, vax_susc_mult)

    update_vax_sim_params(vax_sim, param_modifiers)

    return vax_sim

In [ ]:
# def load_sim_output(folder, npoints, lb=0):
#     scenario_data = pd.DataFrame(columns=UNCERTAINTY_PARAMS+['student_inf_10','student_inf_50','student_inf_90']+\
#                                 ['staff_inf_10', 'staff_inf_50', 'staff_inf_90'])
#     for idx in range(lb, npoints):
#         inf_file = folder + '/list_of_infs_by_group_{}.dill'.format(idx)
#         point_file = folder + '/lhs_point_{}.dill'.format(idx)
#         with open(inf_file, 'rb') as fhandle:
#             inf_matrix = np.array(dill.load(fhandle))
#             student_infxns = np.sum(inf_matrix[:,:-2], axis=1)
#             staff_infxns = np.sum(inf_matrix[:,-2:], axis=1)
#         with open(point_file, 'rb') as fhandle:
#             uncertainty_point = dill.load(fhandle)

#         new_row = dict()
#         for index, col_name in enumerate(UNCERTAINTY_PARAMS):
#             if type(uncertainty_point) == dict:
#                 new_row[col_name] = uncertainty_point[col_name]
#             else:
#                 new_row[col_name] = uncertainty_point[index]
#         new_row['student_inf_10'] = np.quantile(student_infxns, 0.1)
#         new_row['student_inf_50'] = np.quantile(student_infxns, 0.5)
#         new_row['student_inf_90'] = np.quantile(student_infxns, 0.9)
#         new_row['staff_inf_10'] = np.quantile(staff_infxns, 0.1)
#         new_row['staff_inf_50'] = np.quantile(staff_infxns, 0.5)
#         new_row['staff_inf_90'] = np.quantile(staff_infxns, 0.9)
#         new_row['cornell_inf_10'] = np.quantile(student_infxns + staff_infxns, 0.1)
#         new_row['cornell_inf_50'] = np.quantile(student_infxns + staff_infxns, 0.5)
#         new_row['cornell_inf_90'] = np.quantile(student_infxns + staff_infxns, 0.9)
        
#         scenario_data = scenario_data.append(new_row, ignore_index=True)

#     return scenario_data

# def residential_regression_student(scenario_data):
#     columns = scenario_data.columns[0:5]
#     target = 'student_inf_50'
#     X_res = scenario_data[columns]
#     Y_res_outcomes = np.array(scenario_data[[target]])

#     X = add_constant(X_res)
#     model = OLS(Y_res_outcomes,X)
#     results = model.fit()
#     return results

# def residential_regression_staff(scenario_data):
#     columns = scenario_data.columns[0:5]
#     target = 'staff_inf_50'
#     X_res = scenario_data[columns]
#     Y_res_outcomes = np.array(scenario_data[[target]])

#     X = add_constant(X_res)
#     model = OLS(Y_res_outcomes,X)
#     results = model.fit()
#     return results


In [ ]:
# def calculate_pessimistic_scenario(results, q=0.99, beta=1.96):
#     # the keys in dict(results.params) specify whether this is for residential
#     # or virtual vs. residential
#     lr_results = dict(results.params)
#     sd_dict = dict()
#     pess_direction = dict()
#     params = set(lr_results.keys()) - set(['const'])
#     centre_infections = lr_results['const']

#     invquantile = norm.ppf(q)

#     for param in params:
#         sd_dict[param] = (UNCERTAINTY_PARAM_RANGES[param][1] - UNCERTAINTY_PARAM_RANGES[param][0])/(2*beta)
#         centre_infections += np.mean(UNCERTAINTY_PARAM_RANGES[param]) * lr_results[param]

#     sum_squares_Sigma_1 = 0

#     for param in params:
#         sum_squares_Sigma_1 += (lr_results[param]*sd_dict[param]) ** 2

#     for param in params:
#         pess_direction[param] = lr_results[param]*(sd_dict[param])**2 / np.sqrt(sum_squares_Sigma_1)

#     mp_pess_scenario = dict()
#     for param in params:
#         mp_pess_scenario[param] = np.mean(UNCERTAINTY_PARAM_RANGES[param]) + invquantile * pess_direction[param]

#     return mp_pess_scenario

In [ ]:
# LHS_data = load_sim_output('/home/aaj54/group-testing/notebooks/vax_sims/lhs_vax_sims_test_delay:1631153818.0076091', 200)

In [4]:
PARAMS_POST_MOVEIN = {'ug_ga_vax_test_frequency': 2/7, 'ug_ga_unvax_test_frequency': 2/7,
            'ug_other_vax_test_frequency': 1/7, 'ug_other_unvax_test_frequency': 2/7,
            'grad_vax_test_frequency': 1/7, 'grad_unvax_test_frequency': 2/7,
            'employee_vax_test_frequency': 1/7, 'employee_unvax_test_frequency': 2/7,
            'test_delay': 1, 'max_time_pre_ID': 2}

PARAMS_PRE_MOVEIN = {'ug_ga_vax_test_frequency': 1/7, 'ug_ga_unvax_test_frequency': 2/7,
            'ug_other_vax_test_frequency': 1/7, 'ug_other_unvax_test_frequency': 2/7,
            'grad_vax_test_frequency': 1/7, 'grad_unvax_test_frequency': 2/7,
            'employee_vax_test_frequency': 1/7, 'employee_unvax_test_frequency': 2/7,
            'test_delay': 2, 'max_time_pre_ID': 2}

In [5]:
def get_cum_infections(df):
    return df[['cumulative_mild', 'cumulative_severe']].iloc[df.shape[0]-1].sum()


def get_cum_inf_trajectory(df):
    return np.sum(df[['cumulative_mild', 'cumulative_severe']], axis=1)


def run_new_trajectory(sim, T, change_t, override_premovein_params=None):
    sim.reset_initial_state()
    if override_premovein_params != None:
        update_vax_sim_params(sim, override_premovein_params)
    else:
        update_vax_sim_params(sim, PARAMS_PRE_MOVEIN)
        
    for t in range(T):
        sim.step()
        if t == change_t:
            update_vax_sim_params(sim, PARAMS_POST_MOVEIN)

    for single_group_sim in sim.sims:
        single_group_sim.update_severity_levels()

    sim_df = sim.sims[0].sim_df
    for sim in sim.sims[1:]:
        sim_df = sim_df.add(sim.sim_df)
    return sim_df

CHANGE_T = 6 # start of simulation is aug 23, assume changes occur on Aug 29
def run_multigroup_sim(sim, T, override_premovein_params=None):
    run_new_trajectory(sim, T, CHANGE_T, override_premovein_params)
    inf_trajs_by_group = []
    for group in sim.sims:
        df = group.sim_df
        inf_trajs_by_group.append(get_cum_inf_trajectory(df))
    return inf_trajs_by_group


def get_centre_point():
    centre = {}
    for param in UNCERTAINTY_PARAM_RANGES:
        lb, ub = UNCERTAINTY_PARAM_RANGES[param]
        centre[param] = (lb + ub) / 2
    return centre


def run_multiple_trajs(sim, T, n, override_premovein_params=None):
    infs_by_group_list = []
    for _ in range(n):
        infs_by_group = run_multigroup_sim(sim,T, override_premovein_params)
        infs_by_group_list.append(infs_by_group)
    return infs_by_group_list

def get_timestamp():
    return str(time.time()).split('.')[0]

In [12]:
# param_modifiers= PARAMS_PRE_MOVEIN


# nsamples = 5

# # point is either center or pessimistic
# center_dict = get_centre_point()
# center_point = [center_dict['vax_susc_mult'], center_dict['vax_transmission_mult'], center_dict['contacts_per_day_mult'],
#                 center_dict['outside_infection_rate_mult'], center_dict['cases_isolated_per_contact_trace']]
# center_vax_sim = map_lhs_point_to_vax_sim(center_point, param_modifiers)
# center_inf_trajs_by_group = run_multiple_trajs(center_vax_sim, T=112, n=nsamples)


IndexError: list index out of range

In [6]:
def sample_and_save(point_id, save_folder, nsamples=100, T=50):
    prior_point = sample_from_prior()
    prior_sim = map_lhs_point_to_vax_sim(prior_point, param_modifiers)
    inf_trajs_by_group = run_multiple_trajs(prior_sim, T=T, n=nsamples, override_premovein_params=override_params)
    pickle.dump([prior_point, inf_trajs_by_group], open(save_folder+'point_{}'.format(point_id), 'wb'))
    return [prior_point, inf_trajs_by_group]

    
# for i in range(5):
#     # sample from prior
#     prior_point = sample_from_prior()
#     prior_sim = map_lhs_point_to_vax_sim(prior_point, param_modifiers)
#     inf_trajs_by_group = run_multiple_trajs(prior_sim, T=50, n=nsamples, override_premovein_params=override_params)
#     pickle.dump([prior_point, inf_trajs_by_group], open(save_folder+'point_{}'.format(i), 'wb'))

In [7]:

import multiprocessing as mp
import gc
from joblib import Parallel, delayed
import multiprocessing


param_modifiers = PARAMS_PRE_MOVEIN
override_params = PARAMS_POST_MOVEIN
nsamples = 100
T = 50

save_folder = 'samples_for_posterior:{}/'.format(get_timestamp())
os.mkdir(save_folder)

# inputs = np.arange(0.2,.8,.01)
# def processInput(i):
#     gc.collect()
#     spring_calib = SpringCalibration(i)
#     return spring_calib.run_and_score_trajectories(100)

num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(sample_and_save)(i, save_folder, nsamples=nsamples, T=T) for i in range(1000))

ValueError: lam < 0 or lam is NaN

In [17]:
output_folder = 'samples_for_posterior:1633228635'
for i in range(5):
    [prior_point, inf_trajs_by_group] = pickle.load(open(save_folder+'point_{}'.format(i), 'rb'))
    print(prior_point)

[0.485951281822694, 0.531009453873859, 0.8977920248555161, 2.5890114749868807, 1.2607966744668109, 0.001472496499359138]
[0.2574845964104294, 0.8643226554477552, 0.7798119753456287, 2.9627762910006195, 1.4420773570813532, 0.0038408993674479484]
[0.8435744309192734, 0.7429582260272233, 1.4343537968582898, 1.4442732048378428, 0.7812572174117213, 0.004295421386675876]
[0.6948566001322769, 0.7902078026537211, 0.5318534953515408, 1.8138732291999664, 1.0002946042236507, 0.0031307268087275323]
[0.4795219404600336, 0.748161653429301, 2.236011393006872, 0.5418889713026527, 1.5115949369280584, 0.003723865068663194]


In [8]:
base_params, base_group_names, contact_matrix, vax_rates = load_calibrated_params()

In [28]:
# base_params[0]
base_params[2]['daily_outside_infection_p']

6.45e-06